In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import lightgbm as lgb

In [3]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
	sys.path.insert(0, src_path)

from data_loader import load_data
from features import add_time_features, create_target

In [5]:
df = load_data("../data/raw/online_retail_II.csv")

c:\Users\anasm\OneDrive\Desktop\deeplearning\Feature Drift Detection\src\data_loader.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [26]:
X = add_time_features(df)

In [27]:
y = create_target(df , 0.95)

In [28]:
train_mask = df["InvoiceDate"] <= "2010-06-30"
val_mask   = (df["InvoiceDate"] > "2010-06-30") & (df["InvoiceDate"] <= "2010-09-30")
test_mask  = df["InvoiceDate"] > "2010-09-30"


In [29]:
X = X.drop('InvoiceDate', axis=1)
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask], y[val_mask]
X_test,  y_test  = X[test_mask], y[test_mask]

In [30]:
import optuna
import sklearn.metrics

In [31]:
CATEGORICAL_FEATURES = [
    'Country',
    'StockCode'
]

In [32]:
# List of columns identified in the error
# Convert to category type
for col in CATEGORICAL_FEATURES:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')

C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\4100065008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].astype('category')
C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\4100065008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[col] = X_val[col].astype('category')
C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\4100065008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [33]:
# Replace StockCode with its frequency count
enc = X_train['StockCode'].value_counts()
X_train['StockCode_Freq'] = X_train['StockCode'].map(enc)
X_val['StockCode_Freq'] = X_val['StockCode'].map(enc).fillna(0)

# Drop the original high-cardinality column
X_train.drop('StockCode', axis=1, inplace=True)
X_val.drop('StockCode', axis=1, inplace=True)

C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\1472325292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['StockCode_Freq'] = X_train['StockCode'].map(enc)
C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\1472325292.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['StockCode_Freq'] = X_val['StockCode'].map(enc).fillna(0)
C:\Users\anasm\AppData\Local\Temp\ipykernel_2652\1472325292.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [34]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108572 entries, 255103 to 363674
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   Quantity        108572 non-null  int64   
 1   Price           108572 non-null  float64 
 2   Country         108572 non-null  category
 3   Year            108572 non-null  int32   
 4   Month           108572 non-null  int32   
 5   Day             108572 non-null  int32   
 6   Hour            108572 non-null  int32   
 7   min             108572 non-null  int32   
 8   sec             108572 non-null  int32   
 9   StockCode_Freq  108572 non-null  float64 
dtypes: category(1), float64(2), int32(6), int64(1)
memory usage: 5.9 MB


In [35]:
CATEGORICAL_FEATURES = [
    'Country'
]

In [36]:
df_test = X_test.copy()
df_test["target"] = y_test

df_train = X_train.copy()
df_train["target"] = y_train

df_val = X_val.copy()
df_val["target"] = y_val

In [37]:
df_val.head()

,Quantity,Price,Country,Year,Month,Day,Hour,min,sec,StockCode_Freq,target
255103,-1,6.95,United Kingdom,2010,6,30,8,14,0,30.0,0
255104,-2,4.25,United Kingdom,2010,6,30,8,14,0,102.0,0
255105,-2,2.95,United Kingdom,2010,6,30,8,14,0,30.0,0
255106,-1,4.25,United Kingdom,2010,6,30,8,14,0,119.0,0
255107,-2,1.65,United Kingdom,2010,6,30,8,14,0,111.0,0


In [18]:
known_countries = set(df_train["Country"].unique())

def map_country(x):
    return x if x in known_countries else "UNKNOWN"

df["Country"] = df["Country"].apply(map_country)


In [23]:
df.to_csv('../data/processed/data.csv')

In [25]:
df.head()

,Quantity,InvoiceDate,Price,Country,StockCode
0,12,2009-12-01 07:45:00,6.95,United Kingdom,85048
1,12,2009-12-01 07:45:00,6.75,United Kingdom,79323P
2,12,2009-12-01 07:45:00,6.75,United Kingdom,79323W
3,48,2009-12-01 07:45:00,2.10,United Kingdom,22041
4,24,2009-12-01 07:45:00,1.25,United Kingdom,21232


In [38]:
df_test.to_csv("../data/processed/test.csv")
df_val.to_csv("../data/processed/val.csv")
df_train.to_csv("../data/processed/train.csv")

In [1]:
X_train.info()

NameError: name 'X_train' is not defined

In [19]:
# 1. Create Datasets ONCE (Outside the objective function)

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
dvalid = lgb.Dataset(X_val, label=y_val, categorical_feature=CATEGORICAL_FEATURES, reference=dtrain)


def objective(trial):
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "feature_pre_filter": False,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True), # Added this!
        "device": "gpu"
    }

    # 2. Add Early Stopping to save time and prevent overfitting
    gbm = lgb.train(
        param, 
        dtrain, 
        valid_sets=[dvalid],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(period=0) # Keeps the console clean
        ]
    )
    
    preds = gbm.predict(X_val)
    pred_labels = (preds >= 0.5).astype(int)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    return accuracy

In [20]:
study = optuna.create_study(direction="maximize")
study.optimize(objective , n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2026-02-09 18:42:43,422] A new study created in memory with name: no-name-5f460c6c-69cc-4786-abfd-492c55dc1c59


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0522212


[I 2026-02-09 18:44:37,164] Trial 0 finished with value: 0.981901411045205 and parameters: {'lambda_l1': 0.0006486721646784867, 'lambda_l2': 0.013380498742840708, 'num_leaves': 249, 'feature_fraction': 0.5798648360182856, 'bagging_fraction': 0.5524716592243698, 'bagging_freq': 4, 'min_child_samples': 10, 'learning_rate': 0.0017743657170463396}. Best is trial 0 with value: 0.981901411045205.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[294]	valid_0's binary_logloss: 0.000676708


[I 2026-02-09 18:44:57,940] Trial 1 finished with value: 0.9997789485318498 and parameters: {'lambda_l1': 4.648529012961922e-08, 'lambda_l2': 2.073809677338222e-05, 'num_leaves': 173, 'feature_fraction': 0.6476526403798947, 'bagging_fraction': 0.5683294416264257, 'bagging_freq': 3, 'min_child_samples': 76, 'learning_rate': 0.06267811961338399}. Best is trial 1 with value: 0.9997789485318498.


...

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's binary_logloss: 0.0375846


[I 2026-02-09 18:48:43,327] Trial 8 finished with value: 0.9963250193420035 and parameters: {'lambda_l1': 0.00012244613946763307, 'lambda_l2': 1.728219316160374e-06, 'num_leaves': 101, 'feature_fraction': 0.715686594570627, 'bagging_fraction': 0.9614708288934769, 'bagging_freq': 2, 'min_child_samples': 93, 'learning_rate': 0.0019688625555236462}. Best is trial 6 with value: 0.9998065799653686.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[976]	valid_0's binary_logloss: 0.000735851


[I 2026-02-09 18:49:11,209] Trial 9 finished with value: 0.9997697380540103 and parameters: {'lambda_l1': 0.181202085962807, 'lambda_l2': 1.2421179806186093, 'num_leaves': 118, 'feature_fraction': 0.7820415968023464, 'bagging_fraction': 0.49686163364582325, 'bagging_freq': 6, 'min_child_samples': 27, 'learning_rate': 0.03317308669195633}. Best is trial 6 with value: 0.9998065799653686.


Number of finished trials: 10
Best trial:
  Value: 0.9998065799653686
  Params: 
    lambda_l1: 0.0037003441739554025
    lambda_l2: 6.1049012651199e-06
    num_leaves: 111
    feature_fraction: 0.9873427508959625
    bagging_fraction: 0.4976343064175234
    bagging_freq: 1
    min_child_samples: 44
    learning_rate: 0.06553687365646313


In [21]:
from pathlib import Path

Path("../models").mkdir(parents=True, exist_ok=True)



In [22]:
best_params = study.best_params
best_params.update({
    "objective": "binary",
    "metric": "binary_logloss",
    "verbose": -1,
    "boosting_type": "gbdt",
    "device": "gpu",
})

dtrain = lgb.Dataset(X_train, label=y_train, categorical_feature=CATEGORICAL_FEATURES)
gbm = lgb.train(best_params, dtrain)

gbm.save_model("../models/lgbm_optuna.txt")
